<a href="https://colab.research.google.com/github/berkaykarlik/whoIsThatPokemon_CNN/blob/master/Ridge%20regression%20added.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount the Drive

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Import Libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
from tqdm import tqdm



Get the Images

In [9]:
path='/content/gdrive/My Drive/PokemonData'

classes = []
data = []
SIZE = 100


def get_data():
  cnt = 0
  class_no = 0
  for dir in tqdm(os.listdir(path)):
    classes.append(dir)
    dir_path = os.path.join(path,dir)
    for img in tqdm(os.listdir(dir_path)):
      try:
        img_array = cv2.imread(os.path.join(dir_path,img))
        resized_array = cv2.resize(img_array,(SIZE,SIZE))
        data.append([resized_array,class_no])
      except:
        pass
    class_no += 1
get_data()

  9%|▉         | 14/150 [00:05<00:50,  2.69it/s]


Seperate features& labels, convert feature set to numpy array



In [0]:
x = []
y = []
for features,label in data:
    x.append(features)
    y.append(label)

x = np.array(x).reshape(-1, SIZE, SIZE, 3)
y = np.array(y)

Split Data

In [28]:
test_images = []
test_labels = []
train_images = []
train_labels = []


testCnt = 0
for i in range(1,len(y)):
    if y[i] == y[i-1]:
        if testCnt < 5:
            test_images.append(x[i-1])
            test_labels.append(y[i-1])
        else:
            train_images.append(x[i-1])
            train_labels.append(y[i-1])
        testCnt = testCnt + 1
    else:
        testCnt = 0
    
test_images = np.array(test_images).reshape(-1, SIZE, SIZE, 3)
train_images = np.array(train_images).reshape(-1, SIZE, SIZE, 3)
test_labels = np.array(test_labels)
train_labels = np.array(train_labels)


print(train_images.shape,test_images.shape)

(465, 100, 100, 3) (70, 100, 100, 3)


CNN

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras import regularizers

In [0]:
train_images, test_images = train_images / 255.0, test_images / 255.0

In [0]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [0]:
model = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', input_shape=(SIZE, SIZE ,3)),
    Conv2D(64, 3, padding='same', activation='relu',input_shape=(SIZE, SIZE ,3)),
    MaxPooling2D(pool_size=(5, 5),strides=(3, 3), padding='valid'),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(5, 5),strides=(3, 3), padding='valid'),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(150,activity_regularizer=tf.keras.regularizers.l2(0.01))
])

In [0]:
model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'] )
history = model.fit(train_images, train_labels, epochs=150, 
                    validation_data=(test_images, test_labels), callbacks=[callback])

In [0]:
for train, test in kFold.split(X, Y):
    train_evaluate(model, X[train], Y[train], X[test], Y[test])